In [33]:
import duckdb
from sklearn.model_selection import train_test_split

In [4]:
con = duckdb.connect(database=':memory:', read_only=False)

# Loading and preprocessing

In [37]:
# huffpo
q = """
WITH data AS (
SELECT
    *
FROM
    '../../data/raw/news_categories.parquet'
),
total AS (
SELECT COUNT(*) AS total FROM data
),
counts AS (
SELECT category, COUNT(*) AS articles FROM data GROUP BY 1
),
pcts AS (
SELECT category, articles / total.total AS pct FROM counts, total
)
SELECT d.*, REPLACE(REPLACE(SPLIT_PART(d.link, '.com/', 2), '/', ' '), '-', ' ') AS x, category AS y FROM data d JOIN pcts USING (category) WHERE pct > 0.02
LIMIT 1000
"""

huffpo = con.execute(q).fetch_df()

huffpo

,link,headline,category,short_description,authors,date,x,y
0,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,entry funniest tweets cats dogs september 17 2...,COMEDY
1,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,entry funniest parenting tweets_l_632d7d15e4b0...,PARENTING
2,https://www.huffpost.com/entry/dodgers-basebal...,"Maury Wills, Base-Stealing Shortstop For Dodge...",SPORTS,"Maury Wills, who helped the Los Angeles Dodger...","Beth Harris, AP",2022-09-20,entry dodgers baseball obit wills_n_6329feb3e4...,SPORTS
3,https://www.huffpost.com/entry/golden-globes-r...,Golden Globes Returning To NBC In January Afte...,ENTERTAINMENT,"For the past 18 months, Hollywood has effectiv...",,2022-09-20,entry golden globes return nbc_n_6329f151e4b0e...,ENTERTAINMENT
4,https://www.huffpost.com/entry/biden-us-forces...,Biden Says U.S. Forces Would Defend Taiwan If ...,POLITICS,President issues vow as tensions with China rise.,,2022-09-19,entry biden us forces defend taiwan against ch...,POLITICS
...,...,...,...,...,...,...,...,...
995,https://www.huffpost.com/entry/nikola-jokic-ch...,Nikola Jokic Unleashes 'Nasty Cheap Shot' At M...,SPORTS,"The Denver Nuggets star, ejected after his bli...",Ron Dicker,2021-11-09,entry nikola jokic cheap shot markieff morris_...,SPORTS
996,https://www.huffpost.com/entry/joe-biden-jamal...,Joe Biden Promised To Take On Saudi Arabia Ove...,POLITICS,Biden pledged to root his foreign policy in va...,Akbar Shahid Ahmed,2021-11-09,entry joe biden jamal khashoggi saudi arabia_n...,POLITICS
997,https://www.huffpost.com/entry/lance-bass-brit...,Lance Bass Learns He And Britney Spears Are Re...,ENTERTAINMENT,The *NSYNC star said he's always thought of Sp...,Curtis M. Wong,2021-11-09,entry lance bass britney spears ancestry test_...,ENTERTAINMENT
998,https://www.huffpost.com/entry/adam-kinzinger-...,Rep. Adam Kinzinger Reportedly Considered Usin...,POLITICS,"""I barricaded myself in, thinking, ’If this is...",Mary Papenfuss,2021-11-09,entry adam kinzinger jan 6 riot_n_618a058de4b0...,POLITICS


In [30]:
# uci
q = """
SELECT *, REPLACE(REPLACE(SPLIT_PART(URL, '.com/', 2), '/', ' '), '-', ' ') AS x, CATEGORY AS y FROM '../../data/raw/uci_categories.parquet'
"""

uci = con.execute(q).fetch_df()

uci

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP,x,y
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698,business money la fi mo federal reserve plosse...,b
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207,Politics H2EvwJSK2VE6OF7iK1g3PP Feds Charles P...,b
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550,news us open stocks fall after fed official hi...,b
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793,news fed risks falling behind the curve charle...,b
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027,Economy federal reserve charles plosser weathe...,b
...,...,...,...,...,...,...,...,...,...,...
422414,422933,Surgeons to remove 4-year-old's rib to rebuild...,http://www.cbs3springfield.com/story/26378648/...,WSHM-TV,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.cbs3springfield.com,1409229190251,story 26378648 surgeons removed 4 year olds ri...,m
422415,422934,Boy to have surgery on esophagus after battery...,http://www.wlwt.com/news/boy-to-have-surgery-o...,WLWT Cincinnati,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.wlwt.com,1409229190508,news boy to have surgery on esophagus after ba...,m
422416,422935,Child who swallowed battery to have reconstruc...,http://www.newsnet5.com/news/local-news/child-...,NewsNet5.com,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.newsnet5.com,1409229190771,news local news child who swallowed battery to...,m
422417,422936,Phoenix boy undergoes surgery to repair throat...,http://www.wfsb.com/story/26368078/phoenix-boy...,WFSB,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.wfsb.com,1409229191071,story 26368078 phoenix boy undergoes surgery t...,m


In [32]:
# recognasumm

q = """
WITH data AS (
SELECT
    *
FROM
    '../../data/raw/recognasumm.parquet'
),
total AS (
SELECT COUNT(*) AS total FROM data
),
counts AS (
SELECT Categoria AS category, COUNT(*) AS articles FROM data GROUP BY 1
),
pcts AS (
SELECT category, articles / total.total AS pct FROM counts, total
)
SELECT d.*, REPLACE(REPLACE(SPLIT_PART(REPLACE(d.URL, '.com.br', '.com'), '.com/', 2), '/', ' '), '-', ' ') AS x, category AS y FROM data d JOIN pcts ON d.Categoria = pcts.category WHERE pct > 0.02 AND d.URL LIKE 'http%'
"""

rs = con.execute(q).fetch_df()

rs

,index,Titulo,Subtitulo,Noticia,Categoria,Autor,Data,URL,Autor_corrigido,Sumario,__index_level_0__,x,y
0,121981,Bolsonaro autoriza obra de duplicação da BR-4...,Presidente participou de evento na manhã desta...,O presidente Jair Bolsonaro (sem partido) lanç...,Política,Por G1 PR e RPC Foz do Iguaçu — Foz do Iguaçu,27/08/2020 09h43,https://g1.globo.com/pr/oeste-sudoeste/noticia...,Por G1 e RPC Foz do Iguaçu — Foz do Iguaçu,Bolsonaro autoriza obra de duplicação da BR-46...,27048,pr oeste sudoeste noticia 2020 08 27 bolsonaro...,Política
1,46728,Levantamento mostra que ataques cibernéticos ...,País é o 2º na América Latina com mais ataques...,Ingrid Oliveirada CNN O Brasil registrou no pr...,Ciência e Tecnologia,CNN,19/08/2022 às 17:42 | Atualizado 22/08/2022 à...,https://www.cnnbrasil.com.br/tecnologia/levant...,CNN,Levantamento mostra que ataques cibernéticos n...,27052,tecnologia levantamento mostra que ataques cib...,Ciência e Tecnologia
2,8053,Fiocruz: casos de síndrome respiratória aguda ...,Dados mostram desaceleração do crescimento ent...,"Dados do último Boletim InfoGripe da Fiocruz, ...",Saúde,Por O GLOBO — São Paulo,10/03/2023 15h16,https://oglobo.globo.com/saude/medicina/notici...,Por O GLOBO — São Paulo,Fiocruz: casos de síndrome respiratória aguda ...,27039,saude medicina noticia 2023 03 fiocruz casos d...,Saúde
3,26352,Cúpula da Otan redefine geopolítica global; U...,Ouça a análise dos principais fatos da semana ...,"Camila Olivoda CNN Neste episódio, Roberto No...",Brasil,CNN,14/07/2023 às 14:46 | Atualizado 14/07/2023 à...,https://www.cnnbrasil.com.br/nacional/cupula-d...,CNN,Cúpula da Otan redefine geopolítica global; Uc...,27050,nacional cupula da otan redefine geopolitica g...,Brasil
4,133437,"Brasil tem taxas de juros mais altas do mundo,...",As declarações foram dadas horas antes da reun...,O presidente Luiz Inácio Lula da Silva afirmou...,Política,UOL,02/08/2023 15h35,https://noticias.uol.com.br/ultimas-noticias/a...,UOL,"Brasil tem taxas de juros mais altas do mundo,...",27037,ultimas noticias ansa 2023 08 02 brasil tem ta...,Política
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98018,80370,Aumenta o número de imigrantes brasileiros em ...,"Desemprego, situação irregular, falta de prepa...",O número de brasileiros em Portugal que pedira...,Internacional,"Por Luciana Quaresma, RFI",15/02/2023 04h00,https://g1.globo.com/mundo/noticia/2023/02/15/...,Luciana Quaresma,Aumenta o número de imigrantes brasileiros em ...,27,mundo noticia 2023 02 15 aumenta o numero de i...,Internacional
98019,37844,Tropas do Grupo Wagner com mais de 100 veícul...,Imagens de satélite verificadas mostram reboqu...,Paul P. Murphyda CNN O primeiro comboio das fo...,Internacional,CNN,18/07/2023 às 04:01 | Atualizado 18/07/2023 à...,https://www.cnnbrasil.com.br/internacional/tro...,CNN,Tropas do Grupo Wagner com mais de 100 veículo...,24,internacional tropas do grupo wagner com mais ...,Internacional
98020,83970,Presidente da Comissão Europeia visita a Ucrân...,Ursula von der Leyen faz sua terceira visita a...,"A presidente da Comissão Europeia, Ursula von ...",Internacional,Por France Presse,15/09/2022 08h28,https://g1.globo.com/mundo/ucrania-russia/noti...,France Presse,Presidente da Comissão Europeia visita a Ucrân...,16,mundo ucrania russia noticia 2022 09 15 presid...,Internacional
98021,44425,Polícia da Nicarágua proíbe procissão católic...,Relações entre entidade e governo têm sido ten...,Ismael Lopezda Reuters A polícia da Nicarágua ...,Internacional,CNN,12/08/2022 às 17:13,https://www.cnnbrasil.com.br/internacional/pol...,CNN,Polícia da Nicarágua proíbe procissão católica...,11,internacional policia da nicaragua proibe proc...,Internacional


In [38]:
def make_splits(df, seed=20240823):
    X_train, X_test, y_train, y_test = train_test_split(df['x'], df['y'], test_size=0.2, random_state=seed)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=seed)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [39]:
# split each dataset into train (80%), validation (10%) and test (10%) sets
X_train_huffpo, X_val_huffpo, X_test_huffpo, y_train_huffpo, y_val_huffpo, y_test_huffpo = make_splits(huffpo)